# Data Science and Visualization (RUC F2023)

## Miniproject Report: NLP for review evaluation

## Group Members:
* William Grynderup Klindt, RUC, 2023, wgk@ruc.dk
* Luchas Schmidt, RUC, 2023, luchas@ruc.dk
* Jacob Peter Diesel Nielsen, RUC, 2023, jpn@ruc.dk

## Abstract

This project aims to classify the sentiment of reviews using an existing dataset of product reviews from Amazon. We used the dataset to train two supervised models, namely Support Vector Machine and Naive Bayes. Through various data visualization techniques, we investigated and analyzed which model performed better with our dataset and discussed the pros and cons of our results and approaches. We conclude that the Support vector Machine algorithm is generally better due to its flexibility and that we are content with our models accuracy of approximately 70%.

## 0. Setup and Data

Import the modules needed for the project.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import nltk ## If the project can't run, you might have to install NLTK. Please follow the instructions here: https://www.nltk.org/install.html 
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
from nltk.corpus import wordnet

from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english')) 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.svm import SVC

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split 
from sklearn import metrics 

[nltk_data] Downloading package omw-1.4 to /Users/home/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/home/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### Data description:
This dataset contains approximately 568.000 online Amazon reviews for different amazon products. We have minimized the dataset to approximately 10.000 reviews because we seek to prevent computational and runtime issues.
Here is the link for the dataset: https://www.kaggle.com/datasets/jillanisofttech/amazon-product-reviews

## 1. Loading the Data

Our dataset consists of 10.114 rows and 10 columns.

In [2]:
data = pd.read_csv('/Users/luchasschmidt/Git/ML_Bachelor/Reviews_small.csv')
data.shape

FileNotFoundError: [Errno 2] No such file or directory: '/Users/luchasschmidt/Git/ML_Bachelor/Reviews_small.csv'

Below is a sample of the full dataset which consists of rows that display each review in the dataset and each column displays the following:
- ID
- The product ID
- The ID of the author who wrote the review
- The author's username
- Numerator for helpfulness of review
- Denominator for helpfulness of review
- Product rating
- Review time
- A title of the review
- A descriptive text

In [ ]:
data

## 2. Data Preprocessing

Since we downloaded the dataset from Kaggle, they ensured that we received a dataset with zero missing values. 
Next, we have decided to drop some of the columns that we found irrelevant for the project.

These are the columns we removed:

In [ ]:
data = data.drop(labels=['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Time', 'Summary'], axis=1)

As displayed below, the only columns that we found relevant was the "Score" and "Text" column.

In [ ]:
data

To classify reviews as positive, neutral, or negative, we have created a function that evaluates the score of each review. If the review has a score of 3, we consider it to be neutral. If the score is below 3, such as 1 or 2, we consider it to be a negative review. If the score is greater than 3, we consider it to be a positive review.

Below the function, we use a lambda expression to apply the function to each row.

In [ ]:
def classify_sentiment(rating):
    if rating == 3:
        return 1
    elif rating in [1, 2]:
        return 0
    else:
        return 2

data['label'] = data['Score'].apply(lambda x: classify_sentiment(x))

We then print the name of each column in our dataset to verify that the functions worked as intended.

In [ ]:
data.columns

The dataset now consists of reviews with labels to define the positive, negative, or neutral sentiment of each review. 

In [ ]:
data

To ensure that each row contains a value for each column, we use the info() method from Pandas. We also verify that the "label" column only consists of integer values.

In [ ]:
data.info()

### Text cleaning
We seek to clean the text in our dataset to optimize the training model. We found that our text data contains various noise and unwanted characters, such as punctuation, stop words, and special characters, which could potentially affect the quality of the classification process.

We created two functions that are used for lemmatization of each word, meaning that each word should be changed to its root form. The first function, pos_tagger, defines which word class (POS) a given word belongs to. The function takes an NLTK tag as a parameter and returns a Wordnet POS tag. For example, if the input tag is a Noun, it will return a Wordnet Noun tag. If the input tag doesn't match any of the criteria, the function returns None.

Next, we define the function lemmatize_sentence, which is used for lemmatizing sentences. First, we tokenize the sentence with the word_tokenize function from the NLTK library, and secondly, we assign a POS tag to each word with the pos_tag function from the NLTK library. Each POS tag is then converted to a Wordnet POS tag with the beforementioned pos_tagger function. The tokenized words (with their accordingly POS tags) are then stored in a list of tuples.

Lastly, we iterate over the beforementioned list and perform lemmatization of each word with the use of the WordNetLemmatizer and its associated POS tag. If the tag is None (meaning it couldn't assign a tag), the word will be added without lemmatization. If the word got a tag, the word is lemmatized. For both cases, we also make sure that the word is not a stopword. If it is the case, then the word will not be added.

[Reference: Lemmatizer](https://www.geeksforgeeks.org/python-lemmatization-approaches-with-examples/)

In [ ]:
# Define the POS tagger function with modifications for handling unavailable tags
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None  # Use noun as default tag if tag is unavailable''

# Define the function to lemmatize each word in a sentence with its POS tag
def lemmatize_sentence(sentence):
    # Tokenize the sentence and find the POS tag for each token
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))

    # Use our own pos_tagger function to make things simpler to understand
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))

    # Lemmatize each word with its POS tag, ignoring stopwords
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None and word.lower() not in stopword:
            # If there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:
            # Use the tag to lemmatize the token, ignoring stopwords
            if word.lower() not in stopword:
                lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))

    # Join the lemmatized words into a sentence
    return " ".join(lemmatized_sentence)

**Removes any characters that isn't included in the english alphabet.**

In [ ]:
data["Text"] = data['Text'].str.replace('[^a-zA-Z]',' ', regex=True)

We create a new column called "lemma_text" that removes all stop words and lemmatizes each word, meaning that each word should be changed to its root form. For instance, the word "running" should be changed to "run". This is done by calling lemmatize_sencence to the column "Text". 

In [ ]:
data['lemma_text'] = data.Text.apply(lemmatize_sentence)

Below, we showcase the dataset before and after lemmatization.

In [ ]:
data

## 3. Exploratory Data Analysis

In this chapter we aim to present informative visualizations of our dataset that highlight relevant aspects of the data. We will then analyze and interpret these visualizations to gain further insights into the data.

### Histogram showcasing the distribution of the original "Score" column

In [ ]:
plt.hist(data.Score, align='mid', bins=[0.5, 1.5, 2.5, 3.5, 4.5, 5.5])
plt.xticks([1, 2, 3, 4, 5])#, ['Negative', 'Neutral', 'Positive', 'Positive', 'Positive'])
#plt.xlim([0.5, 5.5])
plt.xlabel('Score')
plt.ylabel('Amount')
plt.show()

### Piechart showcasing the distribution of the original "Score" column

In [ ]:
plt.figure(figsize=(5,5))
data['Score'].value_counts().plot.pie(autopct='%1.1f%%', startangle=60)

### Histogram showcasing the distribtuion of the "label" column

In [ ]:
plt.hist(data.label, align='mid', bins=[-0.5, 0.5, 1.5, 2.5])
plt.xticks([0, 1, 2], ['Negative', 'Neutral', 'Positive'])
#plt.xlim([-0.5, 2.5])
plt.xlabel('Sentiment')
plt.ylabel('Amount')
plt.show()

### Piechart showcasing the distribution of the "label" column

In [ ]:
plt.figure(figsize=(4,4))
data['label'].value_counts().plot.pie(autopct='%1.1f%%', startangle=60)

As displayed above, the majority of the labels are valued as positive, and thereby very unbalanced. The different visualizations highlights that our model might be prone to overfitting when trained on this dataset.  

The majority of the positive class can bias the model towards certain patterns or phrases, potentially leading to poor generalization performance on the other two classes. The model may be likely to predict the majority class, even when the input text is from a different class.

To ensure that we train our model properly, without any potential chances of overfitting or general poor performance, we want to balance out our dataset. 

We decided to fix this issue by splitting the score equally among the 5 ratings. Because of computational limits and runtime issues, we have chosen to use 2000 reviews from each rating. This made us use the data from the original dataset containing 568.000 ratings.

In [ ]:
data = pd.read_csv('/Users/luchasschmidt/Git/ML_Bachelor/Reviews.csv')

In [ ]:
amount = 2000

grouped = data.groupby('Score')

data = pd.DataFrame(columns=['Text', 'Score'])

for label, group in grouped:
    selected_group = group.head(amount)
    data = pd.concat([data, selected_group])

First, we applied all of the previous mentioned methods for text cleaning on the large dataset. We will demonstrate how we accomplished this, but due to the dataset's size, the runtime was quite lengthy. Therefore, we only performed the process once and saved the result as a new dataset, which we named "updated_dataframe.csv".

In [ ]:
data = data.drop(labels=['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Time', 'Summary'], axis=1)
data['label'] = data['Score'].apply(lambda x: classify_sentiment(x))
data["Text"] = data['Text'].str.replace('[^a-zA-Z]',' ', regex=True)
data['lemma_text'] = data.Text.apply(lemmatize_sentence)


In [ ]:
data.to_csv('updated_dataframe.csv', index=False)

In [ ]:
data = pd.read_csv('/Users/luchasschmidt/Git/ML_Bachelor/updated_dataframe.csv')

In [ ]:
data

To solve the mentioned issue regarding overfitting we want to balance our dataset. We did that by updating the dataframe so it contains 2000 reviews from each class. The histogram below visualize the new and balanced dataset.

In [ ]:
plt.hist(data.Score, align='mid', bins=[0.5, 1.5, 2.5, 3.5, 4.5, 5.5], width=0.8)
plt.xticks([1, 2, 3, 4, 5])#, ['Negative', 'Neutral', 'Positive', 'Positive', 'Positive'])
plt.xlabel('Score')
plt.ylabel('Amount')
plt.show()

## 4. Data Modelling

In this section of our report we aim to present our approach to data modelling. We first present how to select features for training and validate our model. Lastly, we discuss our findings and evaluate the results.

**We select the features we use in our model.**

In [ ]:
X = np.array(data["lemma_text"]) ## input data, object (string)
y = np.array(data["label"]) ## label, int64 

Instatiating the two machine learning algorithms Naive Bayes and Support Vector Machines (SVM) for classification tasks.
We are currently using both of these machine learning algorithms in our research for our Bachelor project, which lead us to also use the algorithms in this project. We are using Naive Bayes because of its simplicity, fast training time, and computational efficiency. The SVM algorithm is more powerful and gives us the ability to tweak our model in a more customiziable manner. However, we are using the same methods for training both algorithms and SVM should in theory be slower, but more accurate.

[Reference: Naive Bayes](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB)
<br>
[Reference: Support Vector Machine](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)

In [ ]:
nb = MultinomialNB() # Instantierer Naives Bayes
svm = SVC(kernel='linear') # Instantierer SVM, og sætter dens kernal til linear

**Here we split the data into training and test sets. We decide to use 20% of the data for testing.**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)

Here we make use of count vectorizor since we are working with datasets which concists of words, we transform the words into numeric values. We have chosen an ngram range of 1,2, which means we represent word in the sentence as a singe or combination of two words. For instance the sentence "I didn't like this shampoo" is first changed by the ngram range to "I, didn't, like, this, shampoo". Secondly, the sentence is also stored as "I didn't, I like, I this, I shampoo, didn't like, didn't this..." etc. This continious for every combination of every two words.

[Reference: Count Vectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [ ]:
vectCount = CountVectorizer(ngram_range=(1,2))

Training and testing of the before mentioned algorithms. Initially we create a list called models, storing the names of the algorithms we want to modelize.
We then create a function called data_modelling, with two parameters. The first is models, which is the list of algorithms that we want to modelize. The second is vect, and specifics the method that should be used to represent words as numerical values. In our case we only use CountVectorizer as an argument, but other methods for representing the words could be used.

Inside the function we iterate over a for loop according to the length of the models list. For each iteration, meaning for each algorithm, we transform the text to numerical values for both the training and test. We then train the model with the text represented as numerical values and their according label.
After the first model is trained, the accuracy score gets appended to a list, and the next iteration starts. We finally append all the accuracy scores for both models, and return the list as a DataFrame. This gives a DataFrame showcasing the accuracy for the respective models.

In [ ]:
models=[nb, svm]

def data_modelling(models,vect):
    result_table=[]
    for i in range(len(models)):
        X_train_vect = vect.fit_transform(X_train)
        X_test_vect = vect.transform(X_test)
        model = models[i]
        model.fit(X_train_vect, y_train)
        y_pred_class = model.predict(X_test_vect)
        
        data={'Accuracy score':metrics.accuracy_score(y_test, y_pred_class)}
        
        result_table.append(data)  
        
    df = pd.DataFrame(result_table, index =['Naive Bayes', 'Support Vector Machine'])
    return df

In [ ]:
data_modelling(models, vectCount)

### Performance Evaluation

In [ ]:
amount = 200

small_df = pd.DataFrame(columns=['Text', 'Score'])

for label, group in grouped:
    selected_group = group.head(amount)
    small_df = pd.concat([small_df, selected_group])
    
small_df['label'] = small_df['Score'].apply(lambda x: classify_sentiment(x))
small_df['lemma_text'] = small_df.Text.apply(lemmatize_sentence)

X = np.array(small_df["lemma_text"]) ## input data, object (string)
y = np.array(small_df["label"]) ## label, int64 
nb = MultinomialNB() # Instantierer Naives Bayes
svm = SVC(kernel='linear')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)

def data_modelling_2(models,vect):
    result_table=[]
    for i in range(len(models)):
        X_train_vect = vect.fit_transform(X_train)
        X_test_vect = vect.transform(X_test)
        model = models[i]
        model.fit(X_train_vect, y_train)
        y_pred_class = model.predict(X_test_vect)
        
        data={'Accuracy score':metrics.accuracy_score(y_test, y_pred_class)}
        
        result_table.append(data)  
        
    df_2 = pd.DataFrame(result_table, index =['Naive Bayes', 'Support Vector Machine'])
    return df_2

In [ ]:
data_modelling_2(models, vectCount)

## 5. Discussion and analysis

### SVM vs Naive Bayes


We have experimented with the amount of datasets the model should use as training data. In the first model 'data_modelling' the amount of datasets is set to 2000 reviwes for each label, resulting in 10000 datasets in total. Through various tests of our models, we can observe that both models perform evenly with very little difference in accuracy. Accuracy for both of them is around 67-71 which fluctuates for each test. However, the second model that is displayed is using 200 datasets from each label, totalling in 1000 datasets. 'data_modelling_2' is then trained on 20% of the datasets. This results in a decrease of accuracy of around 15% when using both algorithms. 


Due to the results, we believe that the SVM algorithm is the better choice for this project. Our initial plan for this project was to train the two models on the original dataset containing more than half a million datapoints. However, due to hardware limitations this was not possible. Thereby we found the Naive Bayes algorithm as suitable solution for our case.


### Our approach

Our method which classifies reviews as positive, negative or neutral is based on users’ score ranged between 1-5. The method is based on our own considerations of what would be a fair way to classify the sentiment of reviews, which lead us to use a treshold-based-approach: Reviews with a score below 3 are classified as negative, reviews with a score of 3 are classified as neutral, and reviews with a score above 3 are classified as positive.

We believed that a single treshold value for neutral was appropriate for our dataset, but it may not result in a nuanced labelled dataset.

An approach to get a more nuanced classification would be to expand the number of sentiment classes from 3 to 5:

- 1 - Very negative
- 2 - Negative
- 3 - Neutral
- 4 - Postive
- 5 - Very positive

This would capture a wider range of sentiments expressed and a more nuanced classification. This might aswell improve the accuracy of our models, however it may require more resources to implement in our methods and longer compile time.

Another approach we could have implemented is binary classification, where reviews are classified as positive or negative. Reviews with a score of 3 would be removed from the dataset, as they would not fit into either the positive or negative category:

- 1-2 - Negative
- 4-5 - Positive

This approach would have worked well if the goal was to distinguish between positive and negative sentiment of reviews. I would simplify the classification	and probably reduce the complexity of the method. But as a contrast to the before mentioned approach, it would not bring a very nuanced classification or capture any neutral sentiment.

## 6. Conclusion

In this assignment, we analyzed and discussed the use of the Naive Bayes and Support Vector Machine algorithms for machine learning classification tasks.

We found that using both algorithms with a dataset of 10,000 datapoints was almost equally efficient in terms of the accuracy of both models. However, we conclude that the support vector machine algorithm is more sufficient due to its flexibility in working with datasets of different sizes. This is because the accuracy of the second model built on the support vector machine algorithm is about 15% higher when run on a smaller set.

Finally, we would like to shed light on the importance of developing models that contain different algorithms, datasets, sizes, and various approaches to classification. Depending on our approach to this assignment, we are sure we would have seen different results in terms of the model's accuracy. However, we are satisfied with our model's ability to recognize the sentiment of a review, considering our limited data science experience.